In [2]:
import gen3schemadev as g

In [3]:
lipid_schema = g.utils.load_yaml("../tests/gen3_schema/testing/yaml_pass/demographic.yaml")
rule_val = g.validators.rule_validator.RuleValidator(lipid_schema)
rule_val.validate()

Prop keys: ['$ref', 'subjects', 'year_birth', 'month_birth']


In [ ]:
sdfsa

In [2]:
import gen3schemadev as g
rs = g.utils.resolve_schema(schema_path="../tests/gen3_schema/testing/schema_dev_pass.json")
rs = g.utils.resolve_schema(schema_dir="../tests/gen3_schema/examples/yaml")

In [4]:
rs

{'demographic.yaml': {'$schema': 'http://json-schema.org/draft-04/schema#',
  'id': 'demographic',
  'title': 'Demographic',
  'type': 'object',
  'namespace': 'http://commons.heartdata.baker.edu.au/',
  'category': 'clinical',
  'program': '*',
  'project': '*',
  'description': 'Data for the characterization of the patient by means of segementing the population (e.g. characterization by age, sex, or race).',
  'additionalProperties': False,
  'submittable': True,
  'validators': None,
  'systemProperties': ['id',
   'project_id',
   'state',
   'created_datetime',
   'updated_datetime'],
  'links': [{'name': 'subjects',
    'backref': 'demographics',
    'label': 'describes',
    'target_type': 'subject',
    'multiplicity': 'one_to_one',
    'required': True}],
  'required': ['type', 'submitter_id', 'subjects'],
  'uniqueKeys': [['id'], ['project_id', 'submitter_id']],
  'properties': {'type': {'type': 'string'},
   'id': {'term': {'description': 'A 128-bit identifier. Depending on 

In [3]:
type(rs)

dict

In [7]:
g.utils.bundled_schema_to_list_dict(rs)

[{'$schema': 'http://json-schema.org/draft-04/schema#',
  'id': 'demographic',
  'title': 'Demographic',
  'type': 'object',
  'namespace': 'http://commons.heartdata.baker.edu.au/',
  'category': 'clinical',
  'program': '*',
  'project': '*',
  'description': 'Data for the characterization of the patient by means of segementing the population (e.g. characterization by age, sex, or race).',
  'additionalProperties': False,
  'submittable': True,
  'validators': None,
  'systemProperties': ['id',
   'project_id',
   'state',
   'created_datetime',
   'updated_datetime'],
  'links': [{'name': 'subjects',
    'backref': 'demographics',
    'label': 'describes',
    'target_type': 'subject',
    'multiplicity': 'one_to_one',
    'required': True}],
  'required': ['type', 'submitter_id', 'subjects'],
  'uniqueKeys': [['id'], ['project_id', 'submitter_id']],
  'properties': {'type': {'type': 'string'},
   'id': {'term': {'description': 'A 128-bit identifier. Depending on the mechanism used t